In [1]:
# Импортируем нужные нам библиотеки и функции и считываем файл для обработки в pandas
import pandas as pd
from itertools import combinations
from itertools import chain
data = pd.read_csv('/Users/roman_dyatlov/Downloads/movie_bd_v5.csv')

# Предобработка

In [2]:
answers = {}  # Создаём словарь для ответов

data['profit'] = data['revenue'] - data['budget']  # Добавляем колонку "прибыль"

'''Функция, позволяющая вывести единым списком все жанры (с повторениями - столько же, сколько в датасете).
В дальнейшем, через set можно выделить конечный перечень жанров, задействованных в наборе данных:'''


def extract_genres(data_selected):
    list_of_genres = []
    for genre in list(data_selected.genres):
        genre = genre.split("|")

        list_of_genres.append(genre)
    list_of_genres = list(chain(*list_of_genres))
    return list_of_genres


'''Функция, добавляющая к копии датасета столбцы с каждым из жанров с логической переменной,
 отвечающей на вопрос, принадлежит ли данный фильм к указанному жанру:'''


def is_genre_true(data_selected, all_genres):
    data_with_genres = data_selected.copy()
    for genre in all_genres:
        data_with_genres[genre] = data_with_genres.genres.str.contains(genre, na=False)
    return data_with_genres


'''Функция, позволяющая подсчитывать количество фильмов каждого режиссёра
 для определённой выборки из датасета, выдающая Series в качестве результата:'''


def directors_treatment(selected_data):
    selected_data['list_of_directors'] = selected_data['director'].str.split('|')
    list_of_directors = list(chain(*selected_data.list_of_directors))
    s = []
    for d in set(list_of_directors):
        a = selected_data[selected_data.director.str.contains(d, na=False)]
        s.append(a.director.str.contains(d, na=False).count())
    directors_desired = pd.Series(s, index=set(list_of_directors))
    return directors_desired


'''Функция, позволяющая подсчитывать количество фильмов каждого жанра
 для определённой выборки из датасета, выдающая Series в качестве результата. Также можно
 применять произвольную агрегирующую функцию для желаемого подсчёта:'''


def genres_treatment(data_selected, agg_func):
    k = []
    for genre in set(l_o_g):
        a = data_selected[genre].agg(agg_func)
        k.append(a)
    genre_quantities = pd.Series(k, index=set(l_o_g))
    return genre_quantities


'''Функция, позволяющая превращать одну строку, в которой присутствует поле со "сложным" перечнем,
разделённым с помощью знака "|" в несколько строк, в каждой из которых в соответствующем поле остаётся только один 
объект из этого перечня. Функция предполагает работу Вне со всей строкой, а только с двумя выбранными столбцами.
После этого необходимо применить агрегирующую функцию на выбор:'''


def split_explode(selected_data, index_column, values_column, agg_func):
    objects = selected_data[[index_column, values_column]].copy()
    objects[index_column] = objects[index_column].str.split('|')
    objects = objects.explode(index_column)
    objects_desired = objects.groupby(index_column)[values_column].agg(agg_func)
    return objects_desired


l_o_g = extract_genres(data)  # Находим все жанры датасета, не пренебрегая повторениями


data_by_genres = is_genre_true(data, l_o_g)  # Вставляем в копию датасета логические столбцы с каждым жанром

# В основном датасете создаём дополнительную колонку - месяц выпуска фильма, который получаем из колонки "дата выхода".
data['release_month'] = data['release_date'].str.split('/').str[0] 

# В основном датасете создаём дополнительную колонку - длина названия фильма в символах
data['name_characters'] = data.original_title.str.len()

# В основном датасете создаём дополнительную колонку - длина описания в словах
data['description_words'] = data.overview.str.split(' ').str.len()

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [3]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data[data.budget == data.budget.max()])

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5,43,75


ВАРИАНТ 2

In [4]:
# Отображаем запрашиваемую выборку из одной строки через query:
data.query('budget == budget.max()')


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5,43,75


# 2. Какой из фильмов самый длительный (в минутах)?

In [5]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data[data.runtime == data.runtime.max()])

answers['2'] = 'Gods and Generals'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003,-43076064,2,17,48


# 3. Какой из фильмов самый короткий (в минутах)?





In [6]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data[data.runtime == data.runtime.min()])

answers['3'] = 'Winnie the Pooh'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011,-15540000,4,15,52


# 4. Какова средняя длительность фильмов?


In [7]:
# Отображаем среднюю длительность фильмов, записываем ответ:
display(data.runtime.mean())

answers['4'] = '110'


109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [8]:
# Отображаем медианную длительность фильмов, записываем ответ:
display(data.runtime.median())

answers['5'] = '107'


107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [9]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data.query('profit == profit.max()'))

answers['6'] = 'Avatar'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009,2544505847,12,6,28


# 7. Какой фильм самый убыточный? 

In [10]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data.query('profit == profit.min()'))

answers['7'] = 'The Lone Ranger'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,15,68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [11]:
# Отображаем запрашиваемое количество, записываем ответ:
display(data.query('revenue > budget').imdb_id.count())

answers['8'] = '1478'


1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [12]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data.query('release_year == 2008').query('revenue == revenue.max()'))

answers['9'] = 'The Dark Knight'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,15,67


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [13]:
# Отображаем запрашиваемую выборку из одной строки, записываем ответ:
display(data.query('2012 <= release_year <= 2014').query('profit == profit.min()'))

answers['10'] = 'The Lone Ranger'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,15,68


# 11. Какого жанра фильмов больше всего?

In [14]:
'''Подсчитыаем количество фильмов каждого жанра с помощью функции genres_treatment, 
сортируем в порядке убывания, записываем ответ'''

genres_majority = genres_treatment(data_by_genres, sum)
display(genres_majority.sort_values(ascending=False))

answers['11'] = 'Drama'

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

ВАРИАНТ 2

In [15]:
'''Применяем метод "describe" к значению функции "extract genres".
Метод describe содержит в себе строку "top", показывающую наиболее часто встречающийся жанр'''

pd.Series(l_o_g).describe()


count      5107
unique       19
top       Drama
freq        782
dtype: object

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [16]:
'''Используем тело функции "genres_treatment", но в модифицированном виде. Создаём несколько копий
датасета, содержащих фильмы только одного жанра. В каждой копии подсчитываем количество фильмов конкретного
жанра, признанных прибыльными. Результат заносим в список, из которого создаём Series и выводим в качестве ответа.
Записываем правильный ответ в словарь'''

q = []  # Пустой список для количества фильмов жанра
for g in set(l_o_g):
    data_genre = data_by_genres[data_by_genres[g]].copy()
    q.append(data_genre[data_genre['profit'] > 0].imdb_id.count())
most_profit = pd.Series(q, index=set(l_o_g))
display(most_profit.sort_values(ascending=False))

answers['12'] = 'Drama'


Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Foreign              0
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [17]:
'''Создаём сводную таблицу, которая сможет подсчитать кассовые сборы каждого режиссёра. Дополнительно 
отображаем 50 первых строк, чтобы убедиться, что фильмы, режисированные несколькими людьми не влияют на
сборы лидера списка (доходы таких фильмов значительно ниже)'''

dir_reven = data.pivot_table(index='director',
                             values='revenue',
                             margins=True,
                             aggfunc='sum').sort_values(by='revenue', ascending=False)
display(dir_reven.head(50))

answers['13'] = 'Peter Jackson'


,revenue
director,
All,293485091528
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916
Gore Verbinski,3426362290
Steven Spielberg,3386700791
Ridley Scott,3264035909


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [18]:
'''Выделяем выборку только с фильмами в жанре Action. С помощью функции "directors_treatment" подсчитываем 
количество фильмов в этом жанре для каждого режиссёра. Вносим правильный ответ'''

data_action = data[data_by_genres.Action].copy()
directors_action = directors_treatment(data_action)
display(directors_action.sort_values(ascending=False))

answers['14'] = 'Robert Rodriguez'


Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Brian Levant          1
John A. Davis         1
Peter Spierig         1
Jay Russell           1
James Gunn            1
Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [19]:
'''Выделяем выборку только с фильмами, выпущенными в 2012 году. С помощью функции "split_explode" подсчитываем 
доходность фильмов, в которых снимался каждый из актёров. Сортируем в порядке убывания, записываем правильный ответ'''

data_2012 = data.query('release_year == 2012')
revenue_actor = split_explode(data_2012, 'cast', 'revenue', sum)
display(revenue_actor.sort_values(ascending=False))

answers['15'] = 'Chris Hemsworth'


cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [20]:
'''Выделяем выборку только с высокобюджетными фильмами. С помощью функции "split_explode" подсчитываем 
количество фильмов, в которых снимался каждый из актёров. Сортируем в порядке убывания, записываем правильный ответ'''

data_high_budget = data.query('budget > budget.mean()')
exp_actor = split_explode(data_high_budget, 'cast', 'imdb_id', len)
display(exp_actor.sort_values(ascending=False))

answers['16'] = 'Matt Damon'


cast
Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Eddie Murphy         15
Samuel L. Jackson    15
                     ..
Leslie Bibb           1
Leonard Nimoy         1
Lennie James          1
Lena Olin             1
50 Cent               1
Name: imdb_id, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [21]:
'''Выделяем выборку только с фильмами, в которых снимался Николас Кейдж. С помощью функции "genres_treatment" 
подсчитываем количество таких фильмов в определённом жанре. Сортируем в порядке убывания, записываем правильный ответ'''

nick_cage_movies = data_by_genres[data_by_genres.cast.str.contains("Nicolas Cage", na=False)]
nick_cage_genres = genres_treatment(nick_cage_movies, sum)
display(nick_cage_genres.sort_values(ascending=False))

answers['17'] = 'Action'


Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Mystery             3
Animation           3
History             2
Horror              1
War                 1
Romance             1
Foreign             0
Music               0
Western             0
Documentary         0
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [22]:
'''Выделяем выборку только с фильмами, произведёнными компанией "Paramount Pictures". Из этой выборки
с помощью "query" выделяем одну искомую строку. Записываем ответ'''

par_pict_movies = data[data.production_companies.str.contains("Paramount Pictures", na=False)]
less_profit = par_pict_movies.query('profit == profit.min()')
display(less_profit)

answers['18'] = 'K-19: The Widowmaker'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034,7,20,23


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [23]:
'''Создаём сводную таблицу, которая подсчитывает сумму кассовых сборов по годам. Сортируем 
в порядке убывания, записываем правильный ответ'''

success_revenue = data.pivot_table(index='release_year', 
                                   values='revenue', 
                                   margins=True, 
                                   aggfunc='sum').sort_values(by='revenue', ascending=False)
display(success_revenue)

answers['19'] = '2015'


,revenue
release_year,
All,293485091528
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872
2010,21071204961
2009,20261791024
2008,18252781990


# 20. Какой самый прибыльный год для студии Warner Bros?

In [24]:
'''Выделяем выборку только с фильмами, произведёнными компанией "Warner Brothers". Для этой выборки строим 
сводную таблицу, которая подсчитывает суммарные кассовые сборы по годам. Сортируем 
в порядке убывания, записываем правильный ответ'''

warner_bros_movies = data[data.production_companies.str.contains("Warner Bros", na=False)]
success_revenue = warner_bros_movies.pivot_table(index='release_year', 
                                                 values='revenue', 
                                                 margins=True, 
                                                 aggfunc='sum').sort_values(by='revenue', ascending=False)
display(success_revenue)

answers['20'] = '2014'


,revenue
release_year,
All,38636402475
2014,3243064519
2007,3183675217
2008,3050595031
2010,3008712985
2011,2845393682
2003,2745493377
2009,2737954136
2004,2731933725


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [25]:
'''Строим сводную таблицу, подсчитывающую количество фильмов, выходящих в каждый месяц. Сортируем 
в порядке убывания, записываем правильный ответ'''

month_movies = data.pivot_table(index='release_month', 
                                values='imdb_id',
                                margins=True,
                                aggfunc='count').sort_values(by='imdb_id', ascending=False)
display(month_movies)

answers['21'] = 'Сентябрь'


,imdb_id
release_month,
All,1889
9,227
12,190
10,186
8,161
3,156
4,149
6,147
11,146


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [26]:
'''Выделяем выборку из фильмов, выпущенных в летние месяцы. Строим сводную таблицу, подсчитывающую количество
таких фильмов (в "All"). Записываем правильный ответ'''

data_summer = data.query('release_month in ["6","7","8"]')
month_movies = data_summer.pivot_table(index='release_month',
                                       values='imdb_id',
                                       margins=True,
                                       aggfunc='count').sort_values(by='imdb_id', ascending=False)
display(month_movies)

answers['22'] = '450'


,imdb_id
release_month,
All,450
8,161
6,147
7,142


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [27]:
'''Выделяем выборку из фильмов, выпущенных в зимние месяцы. С помощью функции "directors_treatment" считаем 
количество таких фильмов для каждого режиссёра. Сортируем в порядке убывания, записываем правильный ответ '''

data_winter = data.query('release_month in ["1","2","12"]').copy()
directors_winter = directors_treatment(data_winter)
display(directors_winter.sort_values(ascending=False))

answers['23'] = 'Peter Jackson'


Peter Jackson        7
Clint Eastwood       6
Steven Soderbergh    6
Nancy Meyers         4
Martin Scorsese      4
                    ..
Ruben Fleischer      1
Tommy Wirkola        1
Steve Trenbirth      1
Anand Tucker         1
David Dobkin         1
Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [28]:
'''С помощью функции "split_explode" подсчитываем количество символов в названиях фильмов для каждой 
производственной компании. Сортируем в порядке убывания, записываем правильный ответ'''

longest_name = split_explode(data, 'production_companies', 'name_characters', 'mean')
display(longest_name.sort_values(ascending=False))

answers['24'] = 'Four By Two Productions'


production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: name_characters, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [29]:
'''С помощью функции "split_explode" подсчитываем количество слов в описаниях фильмов для каждой производственной
компании. Сортируем в порядке убывания, записываем правильный ответ'''

longest_descr = split_explode(data, 'production_companies', 'description_words', 'mean')
display(longest_descr.sort_values(ascending=False))

answers['25'] = 'Midnight Picture Show'


production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: description_words, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [30]:
'''Сортируем исходный датасет по убыванию по признаку "Рейтинг". Считаем, сколько строк занимает 1% от всех фильмов
и выводим их. Среди вариантов ответа ищем присутствующие фильмы в данном итоговом датасете'''

display(data.sort_values(by='vote_average', ascending=False).head(round(data.imdb_id.count()*0.01)))

answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,name_characters,description_words
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,15,67
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,11/5/2014,8.0,2014,456752480,11,12,35
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,11/14/2014,8.0,2014,219555708,11,18,46
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015,678708609,6,10,112
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015,29401758,10,4,49
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,Leonardo DiCaprio|Jonah Hill|Margot Robbie|Kyl...,Martin Scorsese,EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,Crime|Drama|Comedy,Paramount Pictures|Appian Way|EMJAG Production...,12/25/2013,7.9,2013,292000694,12,23,29
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,10/1/2014,7.9,2014,308330363,10,9,30
1191,tt2024544,20000000,187000000,12 Years a Slave,Chiwetel Ejiofor|Michael Fassbender|Lupita Nyo...,Steve McQueen,The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,Drama|History,Plan B Entertainment|Regency Enterprises|River...,10/18/2013,7.9,2013,167000000,10,16,61
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014,603312399,7,23,28
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Peter Jackson,The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,Adventure|Fantasy|Action,WingNut Films|New Line Cinema,12/1/2003,7.9,2003,1024888979,12,45,45


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [31]:
'''Создаём копию оригинального датасета. Создаём в копии новую колонку, содержащую списки из актёров, снимающихся
в данных фильмах. С помощью "combinations" и лямбда-функции ищем сочетания из набора актёров каждого фильма по парам.
 Подсчитываем, сколько раз повторяются уникальные пары. Записываем в ответ наиболее часто повторяющуюся пару'''
data_pairs = data.copy()
data_pairs['list_of_cast'] = data_pairs.cast.str.split('|')
data_pairs['list_of_cast'] = data_pairs['list_of_cast'].apply(lambda x: tuple(combinations(sorted(x), 2)))
data_pairs = data_pairs.explode('list_of_cast')
frequent_pairs = data_pairs.list_of_cast.value_counts()
display(frequent_pairs)

answers['27'] = 'Daniel Radcliffe & Rupert Grint'


(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Taryn Manning, Terrence Howard)       1
(Julianna Guill, Tom Felton)           1
(Amy Sedaris, Antonio Banderas)        1
(Angie Harmon, Cynthia Stevenson)      1
(Bokeem Woodbine, Jacob Vargas)        1
Name: list_of_cast, Length: 17943, dtype: int64

# Submission

In [32]:
# Проверяем словарь с ответами
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110',
 '5': '107',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [33]:
# Проверяем размер словаря с ответами
len(answers)

27